# Web Scraping with Pandas

Based on the Data Professor youtube video:
https://youtu.be/JUSFaWkAASI

Scrape data from the Wikipedia table with S&P500 and then use Yahoo Finance API to retrieve the last 10 years of stock prices.

### Load Dependencies

In [1]:
import os
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
# Import DB user and password
from api_keys import mysql_hostname
from api_keys import mysql_port
from api_keys import mysql_username
from api_keys import mysql_pass

In [3]:
# MySQL specific connection string
database_name = 'etlprojectdb'
table_price = 'price'
table_companies = 'companies'
database_url = f"mysql+mysqlconnector://{mysql_username}:{mysql_pass}@{mysql_hostname}:{mysql_port}/{database_name}"

In [4]:
# Create the engine
from sqlalchemy import create_engine
engine = create_engine(database_url)
# connection = engine.connect()
# connection.close()

In [5]:
# Test the connection with the database using context manager
with engine.connect() as connection:
    result = connection.execute("select * from companies limit 1")
    for row in result:
        print(row)

### Create Web Scraping

In [6]:
# Wikipedia List of List of S&P 500 companies
url_wikipedia = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Output csv data file name
file_out = os.path.join('List_of_S6P_500_companies.csv')

In [7]:
def load_data(url, table_no):
    html = pd.read_html(url_wikipedia, header=0)
    df = html[table_no]
    return df

In [8]:
companies_list_df = load_data(url_wikipedia, table_no=0)
companies_list_df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [9]:
# Save data as csv
companies_list_df.to_csv(file_out, index=False)

In [10]:
#getting the information for the Comp Table for query

comp_tab_df = companies_list_df

comp_tab_df.drop(columns=['SEC filings', 'Headquarters Location','CIK','Founded'],axis=1, inplace=True)

In [11]:
#remaning columns for loading
comp_tab_df.rename(columns={"Symbol": "comp_tick", "Security": "comp_name", "GICS Sector": "sect_name",
                             "GICS Sub-Industry":"sub_sect_name",
                             "Date first added":"first_trade_date"}, inplace=True)

comp_tab_df.set_index('comp_tick', inplace=True)
comp_tab_df.head(10)

,comp_name,sect_name,sub_sect_name,first_trade_date
comp_tick,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,1976-08-09
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31
ABMD,Abiomed,Health Care,Health Care Equipment,2018-05-31
ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06
ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,2015-08-31
ADBE,Adobe Inc.,Information Technology,Application Software,1997-05-05
AMD,Advanced Micro Devices,Information Technology,Semiconductors,2017-03-20
AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,2015-07-09


In [12]:
companies_list_df['first_trade_date'].iloc[52]

'1983-11-30 (1957-03-04)'

In [13]:
#spit the row that has two dates
companies_list_df['first_trade_date'].iloc[52].split(' ')[0]

'1983-11-30'

In [14]:
#remove the secon date
companies_list_df['first_trade_date'].iloc[52]=companies_list_df['first_trade_date'].iloc[52].split(' ')[0]

In [15]:
#replacing na values with 0
companies_list_df.replace(to_replace="nan", value="0", inplace=False)

companies_list_df.head()

,comp_name,sect_name,sub_sect_name,first_trade_date
comp_tick,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,1976-08-09
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31
ABMD,Abiomed,Health Care,Health Care Equipment,2018-05-31
ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06


In [16]:
# load to sql
# comp_tab_df.to_sql(table_companies, con = engine, if_exists = 'append', chunksize = 50)

# Uncomment this line to upload data to the database
companies_list_df.to_sql(table_companies, con = engine, if_exists = 'append', chunksize = 50)


### Using Yfinance for retrieve stock prices

In [17]:
# List of Stock Symbols
list_symbol_companies = list(companies_list_df.index)

In [18]:
# Get dates for today and ten years ago
today = datetime.now()
# tenYearsAgo = today - timedelta(days=1)
tenYearsAgo = today - timedelta(days=365*10)
today_string = today.strftime("%Y-%m-%d")
tenYearsAgo_string = tenYearsAgo.strftime("%Y-%m-%d")

In [19]:
# Retrieve the stock prices
stock_price = yf.download(list_symbol_companies, start=tenYearsAgo_string, end=today_string)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [20]:
stock_price.head()

Adj Close                                                         \
                    A       AAL        AAP       AAPL ABBV        ABC   ABMD   
Date                                                                           
2011-03-10  29.459438  8.569520  63.120457  10.664602  NaN  32.058064  14.00   
2011-03-11  29.761385  8.654366  63.811100  10.828259  NaN  31.613991  14.03   
2011-03-14  29.433187  8.362119  63.499825  10.876560  NaN  31.272402  14.10   
2011-03-15  28.763657  8.409253  63.636002  10.626457  NaN  31.118685  13.74   
2011-03-16  27.785616  8.060439  63.159370  10.152091  NaN  31.255314  13.59   

                                             ...      Volume              \
                  ABT        ACN       ADBE  ...        XLNX         XOM   
Date                                         ...                           
2011-03-10  18.652031  41.962894  34.860001  ...   7102500.0  25996300.0   
2011-03-11  18.636646  42.118046  34.610001  ...   3920400.0  17607100.0   
2011-03-14  18.544344  42.011898  34.080002  ...   6525100.0  16383200.0   
2011-03-15  18.325140  40.844219  33.040001  ...   6664500.0  23191300.0   
2011-03-16  17.963634  39.995007  32.090000  ...  13807900.0  33396500.0   

                                                                      \
                 XRAY        XRX XYL        YUM        ZBH      ZBRA   
Date                                                                   
2011-03-10  1157700.0  3441154.0 NaN  6367998.0  1470900.0  190200.0   
2011-03-11  1028100.0  3343167.0 NaN  4392222.0   914100.0  124300.0   
2011-03-14  1015400.0  3987369.0 NaN  3231432.0  1055200.0  231400.0   
2011-03-15  2552600.0  4166151.0 NaN  4302641.0  1692600.0  205500.0   
2011-03-16  1638500.0  6874187.0 NaN  5186065.0  2031700.0  347900.0   

                           
                 ZION ZTS  
Date                       
2011-03-10  3578200.0 NaN  
2011-03-11  3494400.0 NaN  
2011-03-14  2587300.0 NaN  
2011-03-15  4546600.0 NaN  
2011-03-16  3924900.0 NaN  

[5 rows x 3030 columns]

In [21]:
stock_price['Close']['AAPL'].head()

Date
2011-03-10    12.381071
2011-03-11    12.571071
2011-03-14    12.627143
2011-03-15    12.336786
2011-03-16    11.786071
Name: AAPL, dtype: float64

In [22]:
stock_price = stock_price.loc['2010-01-01':]

stock_price.head()

Adj Close                                                         \
                    A       AAL        AAP       AAPL ABBV        ABC   ABMD   
Date                                                                           
2011-03-10  29.459438  8.569520  63.120457  10.664602  NaN  32.058064  14.00   
2011-03-11  29.761385  8.654366  63.811100  10.828259  NaN  31.613991  14.03   
2011-03-14  29.433187  8.362119  63.499825  10.876560  NaN  31.272402  14.10   
2011-03-15  28.763657  8.409253  63.636002  10.626457  NaN  31.118685  13.74   
2011-03-16  27.785616  8.060439  63.159370  10.152091  NaN  31.255314  13.59   

                                             ...      Volume              \
                  ABT        ACN       ADBE  ...        XLNX         XOM   
Date                                         ...                           
2011-03-10  18.652031  41.962894  34.860001  ...   7102500.0  25996300.0   
2011-03-11  18.636646  42.118046  34.610001  ...   3920400.0  17607100.0   
2011-03-14  18.544344  42.011898  34.080002  ...   6525100.0  16383200.0   
2011-03-15  18.325140  40.844219  33.040001  ...   6664500.0  23191300.0   
2011-03-16  17.963634  39.995007  32.090000  ...  13807900.0  33396500.0   

                                                                      \
                 XRAY        XRX XYL        YUM        ZBH      ZBRA   
Date                                                                   
2011-03-10  1157700.0  3441154.0 NaN  6367998.0  1470900.0  190200.0   
2011-03-11  1028100.0  3343167.0 NaN  4392222.0   914100.0  124300.0   
2011-03-14  1015400.0  3987369.0 NaN  3231432.0  1055200.0  231400.0   
2011-03-15  2552600.0  4166151.0 NaN  4302641.0  1692600.0  205500.0   
2011-03-16  1638500.0  6874187.0 NaN  5186065.0  2031700.0  347900.0   

                           
                 ZION ZTS  
Date                       
2011-03-10  3578200.0 NaN  
2011-03-11  3494400.0 NaN  
2011-03-14  2587300.0 NaN  
2011-03-15  4546600.0 NaN  
2011-03-16  3924900.0 NaN  

[5 rows x 3030 columns]

### Save data as csv

In [23]:
# Save data as csv
stock_price.to_csv('stock_price.csv', index=False)

In [24]:
clean_stock_price = stock_price

#clean_stock_price.drop(columns=['Adj Close', 'High','Low','Open'],axis=1, inplace=True)
clean_stock_price.drop(columns=['Open'],axis=1, inplace=True)

In [25]:
clean_stock_price.head()

Adj Close                                                         \
                    A       AAL        AAP       AAPL ABBV        ABC   ABMD   
Date                                                                           
2011-03-10  29.459438  8.569520  63.120457  10.664602  NaN  32.058064  14.00   
2011-03-11  29.761385  8.654366  63.811100  10.828259  NaN  31.613991  14.03   
2011-03-14  29.433187  8.362119  63.499825  10.876560  NaN  31.272402  14.10   
2011-03-15  28.763657  8.409253  63.636002  10.626457  NaN  31.118685  13.74   
2011-03-16  27.785616  8.060439  63.159370  10.152091  NaN  31.255314  13.59   

                                             ...      Volume              \
                  ABT        ACN       ADBE  ...        XLNX         XOM   
Date                                         ...                           
2011-03-10  18.652031  41.962894  34.860001  ...   7102500.0  25996300.0   
2011-03-11  18.636646  42.118046  34.610001  ...   3920400.0  17607100.0   
2011-03-14  18.544344  42.011898  34.080002  ...   6525100.0  16383200.0   
2011-03-15  18.325140  40.844219  33.040001  ...   6664500.0  23191300.0   
2011-03-16  17.963634  39.995007  32.090000  ...  13807900.0  33396500.0   

                                                                      \
                 XRAY        XRX XYL        YUM        ZBH      ZBRA   
Date                                                                   
2011-03-10  1157700.0  3441154.0 NaN  6367998.0  1470900.0  190200.0   
2011-03-11  1028100.0  3343167.0 NaN  4392222.0   914100.0  124300.0   
2011-03-14  1015400.0  3987369.0 NaN  3231432.0  1055200.0  231400.0   
2011-03-15  2552600.0  4166151.0 NaN  4302641.0  1692600.0  205500.0   
2011-03-16  1638500.0  6874187.0 NaN  5186065.0  2031700.0  347900.0   

                           
                 ZION ZTS  
Date                       
2011-03-10  3578200.0 NaN  
2011-03-11  3494400.0 NaN  
2011-03-14  2587300.0 NaN  
2011-03-15  4546600.0 NaN  
2011-03-16  3924900.0 NaN  

[5 rows x 2525 columns]

In [26]:
price_df = pd.DataFrame(columns = ["comp_tick","date","close_price","volume","currency"])

In [27]:
row_df = pd.DataFrame(columns = ["comp_tick","close_price","volume","currency"])
count = 505
tick = 1
comp = 0
for row in clean_stock_price.itertuples():

    data = row
    
    date = row[0]
    row_dict=[]
    for i in list_symbol_companies:
        
#         print(data)
        row_dict.append({'comp_tick':list_symbol_companies[comp], 'date':date,'close_price':data[tick], 'volume':  data[count*4+tick],'currency':"USD"})
#         print(row_dict)
        
        tick = 1+tick
        comp = comp+1
    tick = 1
    comp = 0
    price_df = price_df.append(row_dict)

In [28]:
#del price_df
price_df.head()

,comp_tick,date,close_price,volume,currency
0,MMM,2011-03-10,29.459438,6788269.0,USD
1,ABT,2011-03-10,8.569520,12794400.0,USD
2,ABBV,2011-03-10,63.120457,1230700.0,USD
3,ABMD,2011-03-10,10.664602,507539200.0,USD
4,ACN,2011-03-10,NaN,NaN,USD


In [29]:
price_df_new = price_df
price_df_new.set_index('comp_tick', inplace=True)
price_df_new.head(10)

,date,close_price,volume,currency
comp_tick,,,,
MMM,2011-03-10,29.459438,6788269.0,USD
ABT,2011-03-10,8.569520,12794400.0,USD
ABBV,2011-03-10,63.120457,1230700.0,USD
ABMD,2011-03-10,10.664602,507539200.0,USD
ACN,2011-03-10,NaN,NaN,USD
ATVI,2011-03-10,32.058064,1572700.0,USD
ADBE,2011-03-10,14.000000,254700.0,USD
AMD,2011-03-10,18.652031,19423510.0,USD
AAP,2011-03-10,41.962894,2529500.0,USD


In [30]:
price_df_new.head()

,date,close_price,volume,currency
comp_tick,,,,
MMM,2011-03-10,29.459438,6788269.0,USD
ABT,2011-03-10,8.569520,12794400.0,USD
ABBV,2011-03-10,63.120457,1230700.0,USD
ABMD,2011-03-10,10.664602,507539200.0,USD
ACN,2011-03-10,NaN,NaN,USD


In [31]:
price_df.shape

(1270075, 4)

In [32]:
#Load to the sql database

# price_df.to_sql(table_price, con = engine, if_exists = 'append', chunksize = 1000)